In [ ]:
""""
NOTES:
- Use env variables for the end

In [ ]:
#3st Stage - Playout connector
# playout_01
# table name: Channel 

!pip install dnspython
import pandas
from pandas import DataFrame
import os
import pymongo
from pymongo import MongoClient

#Broadcaster data
client = MongoClient("os.environ.get('MONGO_BROADCASTDATA_URI')")
db = client["hybrid-dast"]

    
print("List of collections \n--------------------")

#list collection names
for x in db.list_collection_names():
    print(x)

    
channels = db.channels.find( {} ) #get all documents from channels collection
list_cur = list(channels) #cursor to list
df_channel = DataFrame(list_cur) #cursor to dataframe
    

List of collections 
--------------------
channels
versions
playlists
scripts


In [ ]:
# testing
df_channel

,_id,name,__v,active
0,59f9c0dfbc95750d006b98f5,Dajto,0,False
1,59f9c0c9bc95750d006b98f3,Doma,0,False
2,599ac0c6297e870c00dd0cb5,Markiza,0,False
3,5aba123624af3b0f00984ab9,PRO TV [RO],0,False
4,57336364f9dbf403008dcd3a,Prima,0,False
5,57336364f9dbf403008dcd3b,Prima Cool,0,False
6,5acf3b83ad86850f003eefe8,Prima Krimi,0,False
7,57336364f9dbf403008dcd3c,Prima Love,0,False
8,57336364f9dbf403008dcd3e,Prima Max,0,False
9,57336364f9dbf403008dcd3d,Prima Zoom,0,False


In [ ]:
# removing columns
df_channel.drop(columns=['__v'], inplace=True) 

# renaming columns
df_channel.columns = ['Channel_id', 'Channel_name', 'Active']

In [ ]:
df = df_channel.set_index('Channel_id')
df.head()

,Channel_name,Active
Channel_id,,
59f9c0dfbc95750d006b98f5,Dajto,False
59f9c0c9bc95750d006b98f3,Doma,False
599ac0c6297e870c00dd0cb5,Markiza,False
5aba123624af3b0f00984ab9,PRO TV [RO],False
57336364f9dbf403008dcd3a,Prima,False


In [ ]:
# final result
df.head()

,Channel_name,Active
Channel_id,,
59f9c0dfbc95750d006b98f5,Dajto,False
59f9c0c9bc95750d006b98f3,Doma,False
599ac0c6297e870c00dd0cb5,Markiza,False
5aba123624af3b0f00984ab9,PRO TV [RO],False
57336364f9dbf403008dcd3a,Prima,False


In [ ]:
df.

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 59f9c0dfbc95750d006b98f5 to 5fbe672db5b9900017c168be
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Channel_name  25 non-null     object
 1   Active        25 non-null     bool  
dtypes: bool(1), object(1)
memory usage: 1.0+ KB


In [ ]:
df.to_csv('channel.csv')

In [ ]:
!pip install snowflake-connector-python
import pandas as pd
import snowflake.connector

     |████████████████████████████████| 12.2MB 8.1MB/s 
     |████████████████████████████████| 13.7MB 279kB/s 
     |████████████████████████████████| 2.6MB 55.8MB/s 
     |████████████████████████████████| 112kB 48.1MB/s 
     |████████████████████████████████| 194kB 63.5MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 133kB 37.0MB/s 
     |████████████████████████████████| 337kB 49.2MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 7.2MB 50.6MB/s 
     |████████████████████████████████| 133kB 56.2MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
ERROR: botocore 1.19.53 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
# write to snowflake
conn = snowflake.connector.connect(
                user="os.environ.get('SNOWFLAKE_USER')",
                password="os.environ.get('SNOWFLAKE_PASS')",
                account="os.environ.get('SNOWFLAKE_ACC')",
                database="os.environ.get('SNOWFLAKE_DB')",
                schema="os.environ.get('SNOWFLAKE_SCHEMA')")

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute('USE WAREHOUSE AFTERSPOT_WH')

In [ ]:
cur.execute("put file://{} @{} auto_compress=true".format('channel.csv', 'TEST'))

In [ ]:
cur.execute("COPY INTO DIM_CHANNEL FROM @test/channel.csv.gz FILE_FORMAT=(TYPE=csv field_delimiter=',' skip_header=1)")